In [1]:
import nltk
import fasttext
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.autograd import Variable
from tqdm import tqdm

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
data = pd.read_csv('../corpora/annotated.csv')
data['text'] = data['text'].astype(str)

In [4]:
data['tokens'] = data['text'].apply(nltk.word_tokenize)

In [5]:
# def equalize_lengths(vectors, dst_len=100):
#     if len(vectors) < dst_len:
#         return vectors + [[0] * 100] * (dst_len - len(vectors))
#     elif len(vectors) > dst_len:
#         return vectors[:dst_len]
#     else:
#         return vectors

def get_vectorizer(model, equalized=False, dst_len=100):
    def get_text_repr(tokens):
        return torch.tensor(np.stack([model.get_word_vector(word) for word in tokens]))
    def get_text_repr_eq(tokens):
        vectors = torch.zeros((dst_len, 100))
        vectors[:min(len(tokens), dst_len)] = torch.tensor(np.stack([model.get_word_vector(word) for word in tokens[:dst_len]]))
        return vectors
    if equalized:
        return get_text_repr_eq
    else:
        return get_text_repr

def prepare_data(data, model, equalized=False):
    vectorizer = get_vectorizer(model, equalized)
    X = data['text'].apply(vectorizer)
    y = data['target']
    mask = np.random.rand(len(X)) < 0.8
    X_train = list(X[mask])
    X_test = list(X[~mask])
    y_train = list(y[mask])
    y_test = list(y[~mask])
    return X_train, y_train, X_test, y_test

In [10]:
class MyDataset(Dataset):

    def __init__(self, X, y) -> None:
        super().__init__()
        self.data = list(zip(X, y))

    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)

class MyNet(nn.Module):

    def __init__(self, rlayer: nn.Module) -> None:
        super().__init__()
        self.rnn = rlayer(100, 100, 3, batch_first=True)
        self.reg = nn.Linear(100, 2)
    
    def forward(self, x, x_lengths):
        packed = nn.utils.rnn.pack_padded_sequence(x, x_lengths, batch_first=True, enforce_sorted=False)
        output, _ = self.rnn(packed)
        unpacked, unpacked_len = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        indices = Variable(torch.LongTensor(np.array(unpacked_len) - 1).view(-1, 1)
                                                                       .expand(unpacked.size(0), unpacked.size(2))
                                                                       .unsqueeze(1)
                                                                       .to(device))
        last_encoded_states = unpacked.gather(dim=1, index=indices).squeeze(dim=1)
        return self.reg(last_encoded_states)

class MyConvNet(nn.Module):

    def __init__(self, seq_len=100) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(100, 100, 5),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(100, 100, 3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(100, 100, 2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(1100, 2),
            nn.Softmax()
        )
    
    def forward(self, x, lengths=None):
        return self.net(x)

def my_collate(batch):
    X_batch, y_batch = zip(*batch)
    lengths = [sample.shape[0] for sample in X_batch]
    X_batch = nn.utils.rnn.pad_sequence(X_batch, batch_first=True)
    return X_batch, torch.tensor(y_batch), lengths

def to_dl(X, y, equalized=False):
    if equalized:
        ds = TensorDataset(torch.stack(X), torch.tensor(y))
        dl = DataLoader(ds, 128, shuffle=True, pin_memory=True)
    else:
        ds = MyDataset(X, y)
        dl = DataLoader(ds, 128, shuffle=True, collate_fn=my_collate, pin_memory=True)
    return dl

In [7]:
def validate(model, dl, loss_fn, equalized=False):
    y_pred = []
    y_true = []
    for batch in dl:
        if equalized:
            (X_batch, y_batch) = batch
            lengths = None
        else:
            (X_batch, y_batch, lengths) = batch
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_true.append(y_batch)
        y_pred.append(model(X_batch, lengths))
    y_pred = torch.concat(y_pred)
    y_true = torch.concat(y_true)
    loss = loss_fn(y_pred, y_true)
    y_pred = torch.argmax(y_pred, axis=1).cpu()
    y_true = y_true.cpu()
    return precision_score(y_true, y_pred), recall_score(y_true, y_pred), f1_score(y_true, y_pred), loss

def fit(model, loss_fn, optimizer, train_dl, val_dl, epochs=50, equalized=False, show_metrics=True):
    metrics = {
        'train': {
            'precision': [],
            'recall': [],
            'f1': [],
            'loss': []
        },
        'val': {
            'precision': [],
            'recall': [],
            'f1': [],
            'loss': []
        }
    }
    for epoch in range(epochs):
        for batch in tqdm(train_dl):
            if equalized:
                (X_batch, y_batch) = batch
                lengths = None
            else:
                (X_batch, y_batch, lengths) = batch
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            y_pred = model(X_batch, lengths)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        with torch.no_grad():
            train_prec, train_recall, train_f1, train_loss = validate(model, train_dl, loss_fn, equalized)
            val_prec, val_recall, val_f1, val_loss = validate(model, val_dl, loss_fn, equalized)
            metrics['train']['precision'].append(train_prec)
            metrics['train']['recall'].append(train_recall)
            metrics['train']['f1'].append(train_f1)
            metrics['train']['loss'].append(train_loss.item())
            metrics['val']['precision'].append(val_prec)
            metrics['val']['recall'].append(val_recall)
            metrics['val']['f1'].append(val_f1)
            metrics['val']['loss'].append(val_loss.item())
            if show_metrics:
                print(f'Epoch: {epoch}\ttrain: f1 = {train_f1} loss = {train_loss}\tval: f1 = {val_f1} loss = {val_loss}')
    return metrics


In [8]:
data['target'].value_counts()

0    20959
1    13328
Name: target, dtype: int64

In [11]:
metrics = {}
classifiers = [
    'cnn',
    # 'rnn',
    # 'lstm'
]
for model_path in os.listdir('../dist_models'):
    model_path = '../dist_models/' + model_path
    metrics[model_path] = {}
    for classifier in classifiers:
        equalize = (classifier == 'cnn')
        vectorizer = fasttext.FastText.load_model(model_path)
        X_train, y_train, X_test, y_test = prepare_data(data, vectorizer, equalize)
        train_dl = to_dl(X_train, y_train, equalize)
        test_dl = to_dl(X_test, y_test, equalize)
        if classifier == 'cnn':
            model = MyConvNet().to(device)
        elif classifier == 'rnn':
            model = MyNet(nn.RNN).to(device)
        elif classifier == 'lstm':
            model = MyNet(nn.LSTM).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        print('\n', model_path, classifier,'\n')
        metrics[model_path][classifier] = fit(model, loss_fn, optimizer, train_dl, test_dl, epochs=50, equalized=equalize)

  0%|          | 0/215 [00:00<?, ?it/s]


 ../dist_models/full_m1.bin cnn 



C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 215/215 [00:01<00:00, 176.42it/s]
C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
  0%|          | 0/215 [00:00<?, ?it/s]C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
 12%|█▏        | 25/215 [00:00<00:00, 245.10it/s]

Epoch: 0	train: f1 = 0.21150011962676446 loss = 0.6305745244026184	val: f1 = 0.15226469643430776 loss = 0.6323307752609253


100%|██████████| 215/215 [00:00<00:00, 252.64it/s]
C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  0%|          | 0/215 [00:00<?, ?it/s]C:\Users\pro

...

100%|██████████| 216/216 [00:00<00:00, 255.32it/s]
C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: 49	train: f1 = 0.34686631676443347 loss = 0.5737553238868713	val: f1 = 0.22794330344229097 loss = 0.6524492502212524


  0%|          | 0/214 [00:00<?, ?it/s]C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
  8%|▊         | 17/214 [00:00<00:02, 97.70it/s] 


 ../dist_models/sample_m1.bin cnn 



KeyboardInterrupt: 

In [ ]:
# import json

# with open('metrics_cnn.json', 'w') as f:
#     json.dump(metrics, f)